In [2]:
import random
from utils.gann import Gann
import utils.tflowtools as tft
from utils.caseman import Caseman
import tensorflow as tf

data_set = [
    [[2, 4, 5, 7], [1, 0]],
    [[9, 1, 3, 7], [0, 1]],
]
input_size = len(data_set[0][0])
output_size = len(data_set[0][1])

cman = Caseman(
    lambda: tft.gen_all_parity_cases(input_size),
)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Module 1

## Required Parameters

### Network dimension:
A list of layer sizes. Each element specify the number
of neurons in the corresponding layer.
 
- The first elements specify 
the number of input neurons in the network and should
be equal to the size of case vectors in the data set.
- The subsequent (except the last) elements can be
of arbitrary sizes and specify the sizes (number of 
neurons) of the hidden layers.
- The last element specify the size (number of neurons)
of the output (last hidden layer) and should be equal
to the number of elements in the target vectors in the
data set

#### Format
```python
dims = [2, 4, 3, 4]
# First element is equal to input size
# Last element is equal to output size
```

#### Where to set
```python
Gann(dims, ...)
```

#### Example

In [3]:

dims = [input_size, random.randint(1, 10), random.randint(1, 10), output_size]
gann = Gann(dims, cman=cman)

### Hidden activation function:
A function that represents the activation function for the hidden layers in the network. 

#### Format
```python
tf.nn.relu
tf.nn.relu6
tf.nn.crelu
tf.nn.elu
tf.nn.selu
tf.nn.softplus
tf.nn.softsign
tf.nn.dropout
tf.nn.bias_add
tf.sigmoid
tf.tanh
```

#### Default
```python
tf.nn.relu
```

#### Where to set

```python
Gann(..., hidden_act_func)
```

#### Example

In [9]:
gann = Gann(dims, cman=cman, hidden_act_func=tf.nn.sigmoid)

### Output activation function:
The activation function for the output layer

#### Format
```python
tf.nn.relu
tf.nn.relu6
tf.nn.crelu
tf.nn.elu
tf.nn.selu
tf.nn.softplus
tf.nn.softsign
tf.nn.dropout
tf.nn.bias_add
tf.sigmoid
tf.tanh
```

#### Default
```python
tf.nn.relu
```

#### Where to set:

```python
Gann(..., output_act_func)
```


#### Example

In [6]:
gann = Gann(dims=dims, cman=cman, output_act_func=tf.nn.softmax)

### Cost function
*Also called a loss function.*
Can be either of 
* A string specifying the cost function
* A lambda function that accepts two arguments; (target, output) and returns the error/loss/cost

#### Format
```python
# string
"mse" # mean squared error
"scel" # sigmoid cross entropy (reduced mean)
"smce" # softmax cross entropy (reduce mean)

# function
lambda target, output : tf.reduce_mean(tf.square(target-output))
```

#### Default
```python
tf.reduce_mean(tf.square(self.target - self.output))
```

#### Where to set

```python
Gann(..., cost_func)
```

#### Example

In [6]:
cost_func = lambda target, output: tf.losses.softmax_cross_entropy(onehot_labels=target, logits=output)
gann = Gann(dims=dims, cman=cman, cost_func=cost_func)

### Learning rate
Sets the learning rate of the neural network

#### Format
```python
lrate = 0.1
lrate = 0.3
lrate = 0.01
```

##### Default
```python
lrate = 0.1
```

#### Where to set

```python
Gann(..., lrate)
```

In [7]:
#### Example

In [8]:
lrate = 0.2
gann = Gann(dims=dims, cman=cman, lrate=lrate)

### Initial weight range:
Sets the range for the random chosen initial weights in the network

#### Format
```python
init_weight_range=[-1.0, 1.0]
```

#### Default
```python
init_weight_range=[-.1, .1]
```

#### Where to set:
```python
Gann(..., init_weight_range)
```

### Example

In [9]:
init_weight_range = [-1.0, 1.0]
gann = Gann(dims=dims, cman=cman, init_weight_range=init_weight_range)

### Optimizer
Sets the optimizer function for which will minimize the error. The function must be callable with "minimize(lrate)"

#### Format
```python
tf.train.Optimizer
tf.train.GradientDescentOptimizer
tf.train.AdadeltaOptimizer
tf.train.AdagradOptimizer
tf.train.AdagradDAOptimizer
tf.train.MomentumOptimizer
tf.train.AdamOptimizer
tf.train.FtrlOptimizer
tf.train.ProximalGradientDescentOptimizer
tf.train.ProximalAdagradOptimizer
tf.train.RMSPropOptimizer
```

#### Default
```python
tf.train.GradientDescentOptimizer
```

##### Where to set:
```python
Gann(..., optimizer)
```

### Example

In [10]:
optimizer = tf.train.AdagradOptimizer
gann = Gann(dims=dims, cman=cman, optimizer=optimizer)

### Data source

The data source is passed to the Gann-initializer as a Caseman instance. The Caseman initializer takes four arguments:
* cfunc: a callable function with no arguments which returns a list of cases from the data source in the form (<feature_vector>, <label_vector>)
* vfrac (optional, default: 0): the fraction of the cases to be used for validation
* tfrac (optional, default: 0): the fraction of the cases to be used for testing
* cfrac (optional, default: 0): the fraction of the cases to be used. This can be handy if the data source is very large and you only want to consider a fraction of the cases for training, validation and testing

#### Format
```python
lambda: tft.gen_vector_shift_cases(vlen, count, dir=1, delta=1, density=0.5)
lambda: tft.gen_symvect_cases(vlen, count, label=1)
lambda: tft.gen_anti_symvect_cases(vlen, count, label=0)
lambda: tft.gen_all_one_hot_cases(len, floats=False)
lambda: tft.gen_dense_autoencoder_cases(count, size, dr=(0, 1))
lambda: tft.gen_all_parity_cases(num_bits, double=True)
lambda: tft.gen_vector_count_cases(num, size, drange=(0, 1), random=True, poptarg=True)
lambda: tft.gen_all_binary_count_cases(size, poptarg=True): return gen_vector_count_cases(None, size, random=False, poptarg=poptarg)
lambda: tft.gen_random_line_cases(num_cases, dims, min_lines=1, min_opens=1, bias=1.0, mode='classify', line_item=1, background=0, flat=True, floats=False)
lembda: tft.gen_segmented_vector_cases(vectorlen, count, minsegs, maxsegs, poptargs=True)
```

#### Where to set
```python
cman = Caseman(cfunc, vfrac, tfrac, cfrac)
gann = Gann(dims=dims, cman=cman)
```

### Example

In [4]:
cfunc = lambda: tft.gen_all_parity_cases(num_bits=8)

cman = Caseman(cfunc=cfunc)

gann = Gann(dims=dims, cman=cman)

### Case fraction:
The fraction of the whole data set to consider as cases for training, validation and testing.

#### Where to set:
```python
Caseman(..., cfrac)
```
#### Format
```python
0.8
0.5
```

#### Default
```python
1.0 # the whole data set
```


### Validation fraction:
The fraction of the cases to be used for validation.

#### Where to set:
```python
Caseman(..., vfrac)
```

##### Format:
```python
0.1
0.2
```

##### Default:
```python
0.0
```


### Validation interval
Number of training minibatches between each validation test

#### Format
```python
vint = 10
vint = 100
vint = 1000
```

#### Default
```python
None
```

#### Where to set
```python
vint = 100
gann = Gann(vint=vint)
```

#### Example

In [5]:
vint = 100
gann = Gann(dims=dims, cman=cman, vint=vint)